<a href="https://colab.research.google.com/github/gibranfp/CursoAprendizajeProfundo/blob/2023-1/notebooks/1d_einsum_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch as th
from torch import nn

## `einsum`
PyTorch cuenta con la función `einsum`, que es una forma compacta y flexible de expresar diversas operaciones tensoriales. Esta función recibe como argumento una cadena de símbolos que expresa operaciones sobre uno o más tensores. Esta expresión se basa en la [notación de Einstein](https://en.wikipedia.org/wiki/Einstein_notation) y tiene la siguiente forma

```
D = th.einsum("IA,IB,IC->ID", A, B, C)
```
donde `IA`, `IB`, `IC` e `ID` son secuencias de índices de los tensores `A`, `B`, `C` y `D`, respectivamente.

Para entender cómo funciona `einsum` supongamos que queremos realizar el siguiente producto punto:

$$
\overbrace{
\begin{pmatrix}
  9 & 7 & 3\\ 
  2 & 8 & 4
\end{pmatrix}}^A
\times
\overbrace{
\begin{pmatrix}
  9 & 9 & 6 & 5 \\ 
  9 & 1 & 4 & 2\\
  8 & 7 & 1 & 3
\end{pmatrix}}^B
=
\overbrace{
\begin{pmatrix}
  168 & 109 & 85 & 68\\ 
  122 & 54 & 48 & 38
\end{pmatrix}}^C
$$

Podemos expresar el valor de cada elemento de la matriz resultante como:
$$
C_{ij} = \sum_{k}^3 A_{ik} \cdot B_{kj}
$$

En la notación de Einstein los símbolos de suma se omiten, por lo que esta expresión sería:

$$
C_{ij} = A_{ik} \cdot B_{kj}
$$

Muchas veces, es útil pensar en cómo se realizaría la operación simplemente con ciclos `for`. Para este caso:

```python
for i in range(2):
  for j in range(4):
    C[i,j] = sum([A[i,k] * B[k,j] for k in range(3)])
```
En `einsum` se usa esta notación para definir el tensor de salida, que para nuestro ejemplo sería:



In [2]:
A = th.tensor([[9,7,3], [2,8,4]])
B = th.tensor([[9,9,6,5], [9,1,4,2], [8,7,1,3]])
print(f'einsum: {th.einsum("ik,kj->ij", A, B)}')
print(f'A @ B: {A @ B}')

einsum: tensor([[168, 109,  85,  68],
        [122,  54,  48,  38]])
A @ B: tensor([[168, 109,  85,  68],
        [122,  54,  48,  38]])


A $i$ y a $j$ se les conoce como índices libres y son los que se ponen en el tensor de salida para definir su forma y valores en términos de los tensores de entrada. Por su parte, $k$ se conoce como índice de suma y se pone solamente en los tensores de entrada para especificar los elementos que se van a sumar.

Para el producto punto, producto exterior y la multiplicación matriz-vector:

In [3]:
C = th.tensor([[1., 2., 3.], [1., 2., 3.]])
x1 = th.tensor([1., 2., 3.])
x2 = th.tensor([1., 2., 3.])
print(f'Producto punto: {th.einsum("i,i->", x1, x2)}')
print(f'Producto exterior: {th.einsum("i,j->ij", x1, x2)}')
print(f'Multiplicación matriz-vector: {th.einsum("ij,j->i", C, x1)}')

Producto punto: 14.0
Producto exterior: tensor([[1., 2., 3.],
        [2., 4., 6.],
        [3., 6., 9.]])
Multiplicación matriz-vector: tensor([14., 14.])


Producto con transpuesta

In [4]:
D = th.rand((400, 1000))
E = th.rand((200, 1000))

print(f'Transpuesta de  D: {th.einsum("ji->ij", D)}')
print(f'Producto de D con transpuesta de E: {th.einsum("ik,jk->ij", D, E)}')

Transpuesta de  D: tensor([[0.3410, 0.7847, 0.8038,  ..., 0.4330, 0.4495, 0.1376],
        [0.8568, 0.6267, 0.6338,  ..., 0.0207, 0.7869, 0.2014],
        [0.3019, 0.9199, 0.4016,  ..., 0.9133, 0.9401, 0.9685],
        ...,
        [0.7795, 0.0561, 0.3957,  ..., 0.9613, 0.3996, 0.3911],
        [0.6834, 0.3947, 0.8615,  ..., 0.2543, 0.5774, 0.5915],
        [0.3250, 0.7353, 0.6168,  ..., 0.5994, 0.6221, 0.8881]])
Producto de D con transpuesta de E: tensor([[252.8139, 264.1091, 252.2740,  ..., 257.2824, 259.7156, 257.6115],
        [248.8030, 257.5115, 248.7957,  ..., 252.0943, 256.9398, 254.6261],
        [256.2570, 261.2106, 250.1201,  ..., 255.7995, 252.3157, 257.5032],
        ...,
        [256.5848, 255.2772, 252.0509,  ..., 250.9626, 256.1165, 256.6591],
        [252.6007, 253.6698, 247.5227,  ..., 251.4182, 249.2036, 251.3750],
        [252.1222, 252.0891, 248.0955,  ..., 253.0552, 257.8606, 249.6725]])


Multiplicación por columna/fila

In [5]:
G = th.rand((5, 11))
H = th.rand((5, 11))
print(f'Columna: {th.einsum("ij,ij->j", G, H)}')
print(f'Fila: {th.einsum("ij,ij->i", G, H)}')

Columna: tensor([1.4510, 0.6140, 0.8891, 0.3801, 0.7448, 1.9619, 0.6529, 2.3836, 1.6405,
        1.2919, 1.3124])
Fila: tensor([1.5682, 3.5801, 2.4981, 2.2494, 3.4264])


La suma de todos los elementos de un tensor se puede expresar en `einsum` como:

In [6]:
print(f'Suma vector: {th.einsum("i->", th.ones((3)))}') # no tiene índices libres
print(f'Suma matriz: {th.einsum("ij->", th.ones((3,3)))}') # no tiene índices libres
print(f'Suma tensor: {th.einsum("ijk->", th.ones((3,3,3)))}') # no tiene índices libres

Suma vector: 3.0
Suma matriz: 9.0
Suma tensor: 27.0


Para el reordenamiento de ejes

In [7]:
print(f'Reordenamiento de ejes: {th.einsum("ijk->kji", th.ones((5, 4, 3))).shape}') # no tiene índices de suma

Reordenamiento de ejes: torch.Size([3, 4, 5])


Extraer elementos de la diagonal

In [8]:
F = th.diag(th.tensor([1., 2., 3.]))
print(f'Elementos de diagonal: {th.einsum("ii->i", F)}')
print(f'Elementos de diagonal: {th.einsum("ii->", F)}')

Elementos de diagonal: tensor([1., 2., 3.])
Elementos de diagonal: 6.0


## Ejercicios
- Multiplica elemento a elemento dos matrices de $500 \times 1000$ y suma las filas de la matriz resultante.
- Calcula y despliega el producto de cada matriz de $400 \times 1000$ en un tensor aleatorio $A$ de tamaño $500 \times 400 \times 1000$ con la transpuesta de un tensor aleatorio $B$ de tamaño $200 \times 1000$. Concatena el resultado a un tensor aleatorio $C$ con el mismo número de columnas que el resultado de la operación anterior. Compara los tiempos de ejecución de esta operación con usar `th.reshape` y `th.matmul`. 